# Exporing Ebay Car Sales Data

In this project, we will work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the Gernam eBay website.

The original dataset and data dictionary can be found [here](https://www.kaggle.com/orgesleka/used-cars-database/data), but the version used in this project has been modified as follows:

* 50,000 data points were sampled from the full dataset of more than 370,000 observations
* The dataset was "dirtied" a bit to more closely resemble what we would expect from a scraped dataset. The original version of the dataset was cleaned before being uploaded to Kaggle.

The purpose of this project is to clean the data and analyze the included used car listings.

In [1]:
import numpy as np
import pandas as pd

In [2]:
autos = pd.read_csv("autos.csv", encoding="Latin-1")

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [5]:
autos.isnull().sum()

dateCrawled               0
name                      0
seller                    0
offerType                 0
price                     0
abtest                    0
vehicleType            5095
yearOfRegistration        0
gearbox                2680
powerPS                   0
model                  2758
odometer                  0
monthOfRegistration       0
fuelType               4482
brand                     0
notRepairedDamage      9829
dateCreated               0
nrOfPictures              0
postalCode                0
lastSeen                  0
dtype: int64

We can see that several columns have missing values - "vehicleType", "gearbox", "model", "fuelType", and "notRepairedDamage". Many of the observations appear to be written in German. Additionally, the column names are written in camelcase rather than snakecase. We will fix these issues below.

First, we will convert the column names to snakecase and reword some of the column names to make them more descriptive.

In [6]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [7]:
new_columns = ["date_crawled", "name", "seller", "offer_type", "price", "abtest", "vehicle_type", "registration_year",
               "gearbox", "power_ps", "model", "odometer", "registration_month", "fuel_type", "brand",
               "unrepaired_damage", "ad_created", "number_of_pictures", "postal_code", "last_seen"]
autos.columns = new_columns
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


After updating the names of some columns to be more descriptive and changing the column names from camelcase to snakecase, the column names are much easier to read and understand.

Now, let's do some basic data exploration to determine what other cleaning tasks need to be done.

In [8]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 17:37:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


* It appears that the 'price' and 'odometer' columns are numeric values stored as text. We will need to convert these columns to numeric data types.
* The 'seller' and 'offer_type' columns seem like they could be dropped from the dataframe - each contains only 2 unique values and all but 1 row have the same value.
* The 'registration_year', 'registration_month', 'number_of_pictures', and 'postal_code' columns need to be investigated further because of the number of unique values was shown as NaN in the describe() function.

First, let's investigate 'registration_year' and 'registration_month'

In [9]:
print(autos["registration_year"].unique().shape)
autos["registration_year"].unique()

(97,)


array([2004, 1997, 2009, 2007, 2003, 2006, 1995, 1998, 2000, 2017, 2010,
       1999, 1982, 1990, 2015, 2014, 1996, 1992, 2005, 2002, 2012, 2011,
       2008, 1985, 2016, 1994, 1986, 2001, 2018, 2013, 1972, 1993, 1988,
       1989, 1967, 1973, 1956, 1976, 4500, 1987, 1991, 1983, 1960, 1969,
       1950, 1978, 1980, 1984, 1963, 1977, 1961, 1968, 1934, 1965, 1971,
       1966, 1979, 1981, 1970, 1974, 1910, 1975, 5000, 4100, 2019, 1959,
       9996, 9999, 6200, 1964, 1958, 1800, 1948, 1931, 1943, 9000, 1941,
       1962, 1927, 1937, 1929, 1000, 1957, 1952, 1111, 1955, 1939, 8888,
       1954, 1938, 2800, 5911, 1500, 1953, 1951, 4800, 1001], dtype=int64)

In [10]:
print(autos["registration_month"].unique().shape)
autos["registration_month"].unique()

(13,)


array([ 3,  6,  7,  4,  8, 12, 10,  0,  9, 11,  5,  2,  1], dtype=int64)

* A few values in the "registration_year" column look suspect - 4500, 5000, 4100, 9996, 9999, 6200, 1800, 9000, 1000, 1111, 8888, 2800, 4800, 1001
* The "registration_month" column has 13 unique values rather than the expected 12. It appears some rows have a value of 0 for registration month.

Next, let's investigate the "number_of_pictures" and "postal code" fields

In [11]:
print(autos["number_of_pictures"].unique().shape)
autos["number_of_pictures"].unique()

(1,)


array([0], dtype=int64)

In [12]:
print(autos["postal_code"].unique().shape)
autos["postal_code"].unique()

(7014,)


array([79588, 71034, 35394, ..., 34317, 97502, 84385], dtype=int64)

* The "number_of_pictures" column appears to contain nothing but 0 for all entries. This column can be dropped.
* The "postal_code" appears to not have any issues - there are over 7,000 unique values and we don't know enough about German postal codes to know if any values are invalid.

Next, we will clean the "price" and "odometer" columns by removing non-numeric characters and converting the columns to a numeric data type.

In [13]:
autos["price"].head(10)

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
5    $7,900
6      $300
7    $1,990
8      $250
9      $590
Name: price, dtype: object

In [14]:
autos["price"] = (autos["price"]
                  .str.replace("$", "")
                  .str.replace(",", "")
                  .astype(float)
                 )

In [16]:
autos["price"].head(10)

0    5000.0
1    8500.0
2    8990.0
3    4350.0
4    1350.0
5    7900.0
6     300.0
7    1990.0
8     250.0
9     590.0
Name: price, dtype: float64

In [17]:
autos["odometer"].head(10)

0    150,000km
1    150,000km
2     70,000km
3     70,000km
4    150,000km
5    150,000km
6    150,000km
7    150,000km
8    150,000km
9    150,000km
Name: odometer, dtype: object

In [18]:
autos["odometer"] = (autos["odometer"]
                  .str.replace("km", "")
                  .str.replace(",", "")
                  .astype(float)
                 )

In [19]:
autos["odometer"].head(10)

0    150000.0
1    150000.0
2     70000.0
3     70000.0
4    150000.0
5    150000.0
6    150000.0
7    150000.0
8    150000.0
9    150000.0
Name: odometer, dtype: float64

In [20]:
autos.rename({"odometer":"odometer_km"}, axis=1, inplace=True)

Now, let's take a closer look at the "price" and "odometer_km" columns.